In [ ]:
!pip install tensorflow


In [ ]:
!pip install --upgrade pip


In [ ]:
!conda install -c anaconda tensorflow-gpu



In [ ]:
!pip cache purge
!pip install tensorflow-gpu


In [ ]:
!pip install tensorflow-gpu

In [ ]:
import tensorflow as tf
print("GPU Available:", tf.config.list_physical_devices('GPU'))


In [ ]:
!pip install deap

In [ ]:
pip install opencv-python

In [ ]:
from deap import base, creator, tools, algorithms

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16  # Import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling3D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from google.colab import drive

# Mount Google Drive (this will prompt you to authorize access)
drive.mount('/content/drive')

# Define the dimensions of your 3D data
width, height, depth = (224, 224, 64)  # Adjust dimensions as needed

# Load and preprocess 3D data from Google Drive
data_folder = "/content/drive/MyDrive/Normalization_output_file2"  # Replace with your Google Drive folder path
output_folder = "/content/drive/MyDrive/Models"  # Replace with the path to save trained models on Google Drive
num_epochs = 20  # Adjust the number of epochs as needed
batch_size = 1  # Adjust batch size as needed

# Create a mapping from class names to numeric labels
class_name_to_label = {
    'VeryMildDemented': 0,
    'MildDemented': 1,
    'ModerateDemented': 2,
    'NonDemented': 3
}

# Initialize empty lists to store data and labels
data = []
labels = []

# Load data from the subfolders of your data folder
for class_folder in os.listdir(data_folder):
    if os.path.isdir(os.path.join(data_folder, class_folder)):
        class_path = os.path.join(data_folder, class_folder)
        class_label = class_name_to_label.get(class_folder, -1)  # Get the numeric label from the mapping

        if class_label != -1:
            for file_name in os.listdir(class_path):
                if file_name.endswith(".npy"):
                    file_path = os.path.join(class_path, file_name)
                    image_data = np.load(file_path)
                    # Check the number of channels in the image
                    num_channels = image_data.shape[2] if len(image_data.shape) > 2 else 1
                    data.append(image_data)
                    labels.append(class_label)

# Convert data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Split the data into training and validation sets (80% - 20%)
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define data generators with data augmentation for 3D volumes
def data_generator(data, labels, batch_size, train=True):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode="nearest",
        rescale=1.0 / 255
    )

    while True:
        indices = np.arange(len(data))
        if train:
            np.random.shuffle(indices)

        for start in range(0, len(data), batch_size):
            end = min(start + batch_size, len(data))
            batch_indices = indices[start:end]
            batch_data = data[batch_indices]
            batch_labels = labels[batch_indices]

            if train:
                yield datagen.flow(batch_data, batch_labels, batch_size=batch_size, shuffle=False)
            else:
                yield batch_data, batch_labels

# Create the VGG16-based 3D model
num_classes = len(np.unique(train_labels))

def create_model(learning_rate, num_channels):
    input_shape = (width, height, depth, num_channels)
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)  # Use VGG16
    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling3D())
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    optimizer = Adam(lr=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Rest of the code remains the same
# Define a function to evaluate the model's fitness
def evaluate_model(individual):
    learning_rate = individual[0]
    num_channels = train_data.shape[4]  # Get the number of channels from the training data
    model = create_model(learning_rate, num_channels)

    # Define data generators for this epoch
    train_generator = data_generator(train_data, train_labels, batch_size, train=True)
    val_generator = data_generator(val_data, val_labels, batch_size, train=False)

    # Train the 3D model with callbacks for this epoch
    model.fit(train_generator,
              epochs=num_epochs,
              validation_data=val_generator,
              callbacks=[LearningRateScheduler(lr_schedule), early_stopping],
              verbose=0)

    # Evaluate the model on the validation data for this epoch
    _, val_accuracy = model.evaluate(val_generator, verbose=0)

    # Minimize the negative validation accuracy
    return -val_accuracy

# Profile the code using TensorFlow Profiler
from tensorflow.python.profiler import profiler_client

def profile_code():
    with profiler_client.trace('/content/drive/MyDrive/profile_output'):
        # Your existing code here
        pass

profile_code()

# Create the DEAP toolbox
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, 1e-5, 1e-3)  # Learning rate range
toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.attr_float,), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Define the genetic operators
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

# Create an initial population of 10 individuals
population = toolbox.population(n=6)

# Create the hall of fame to store the best individual
hall_of_fame = tools.HallOfFame(1)

# Create the statistics object
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("max", np.max)
stats.register("avg", np.mean)

# Run the genetic algorithm
NGEN = 8  # Number of generations
CXPB = 0.7  # Crossover probability
MUTPB = 0.2  # Mutation probability

# Create a logbook to keep track of the evolution
logbook = tools.Logbook()
logbook.header = ["gen", "evals", "max", "avg"]

for gen in range(NGEN):
    # Evaluate the entire population
    fitnesses = map(evaluate_model, population)
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit

    # Update the hall of fame with the best individual
    hall_of_fame.update(population)

    # Clone the population
    offspring = list(map(toolbox.clone, population))

    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if np.random.rand() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if np.random.rand() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluate the offspring
    fitnesses = map(evaluate_model, offspring)
    for ind, fit in zip(offspring, fitnesses):
        ind.fitness.values = fit

    # Replace the old population by the offspring
    population[:] = offspring

    # Record statistics
    record = stats.compile(population)
    logbook.record(gen=gen, evals=len(population), **record)
    print(logbook.stream)

# Get the best individual and its fitness value from the hall of fame
best_ind = hall_of_fame[0]
best_fitness = best_ind.fitness.values[0]

# Print the best individual and its fitness value
print("Best Individual:")
print("Learning Rate =", best_ind[0])
print("Best Fitness =", best_fitness)

# Create and save the best model with the optimized learning rate
best_learning_rate = best_ind[0]
num_channels = train_data.shape[4]  # Get the number of channels from the training data
best_model = create_model(best_learning_rate, num_channels)

# Train the best model on the entire dataset using generators
best_train_generator = data_generator(train_data, train_labels, batch_size)
best_val_generator = data_generator(val_data, val_labels, batch_size, train=False)

best_model.fit(best_train_generator, epochs=num_epochs, verbose=1)
best_model.save(os.path.join(output_folder, "Best_VGG16_model_3d.h5"))

# Evaluate the best model on the validation data
val_loss, val_accuracy = best_model.evaluate(best_val_generator, verbose=0)

# Print validation loss and accuracy of the best model
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

# Clear any unnecessary variables to release memory
del data
del labels
del train_data
del val_data
